In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download("all")
from nltk.stem import *
from nltk.tokenize import *
from sklearn.feature_extraction.text import *

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/amith/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/amith/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |

In [95]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [96]:
df = pd.read_excel("IELTS_Success_Stories_Dataset.xlsx")

In [97]:
df

,Candidate,Location,Profession,Study Duration (months),IELTS Score (Overall),Key Strategies,Education Level,Age,Target Country,English Proficiency (Preparation),...,Preferred Learning Resources,Application Status,Job Offer Received,Additional Certifications,Volunteer Experience,Language Fluency,Internship Experience,Relevant Skills,Recommendations,Networking Efforts
0,Aisha,Karachi,Grad Student,3,8.500,"Practice, forums, tutoring",Master's,24,Canada,Advanced,...,"Online courses, study guides",Under Process,No,TESOL,Yes,"Urdu, English",Research Assistant,"Data Analysis, Teaching",Strong,"LinkedIn, Local Events"
1,Ahmed,Lahore,IT Professional,4,7.500,"Balanced schedule, time management, speaking p...",Bachelor's,29,Australia,Intermediate,...,"Online forums, practice tests",Approved,Yes,PMP,No,"Urdu, English",IT Consultant,"Project Management, Programming",Positive,Online Communities
2,Fatima,Islamabad,School Teacher,2,9.000,"Intensive course, online resources, mock inter...",Master's,32,New Zealand,Advanced,...,"Study guides, language exchange",Under Process,No,CELTA,Yes,"Urdu, English",Curriculum Developer,"Lesson Planning, Educational Tech",Positive,Professional Groups
3,Ali,Peshawar,IT Professional,5,8.000,"Focused study, workshops, peer feedback",Bachelor's,27,USA,Advanced,...,"Mock exams, speaking clubs",Rejected,No,AWS Certified Solutions Architect,No,"Urdu, English",Software Developer,"Cloud Computing, Programming",Strong,Online Platforms
4,Saad,Lahore,Eng. Student,6,8.250,"Consistent practice, private tutor, speaking c...",Bachelor's,22,UK,Advanced,...,"Online tutorials, speaking practice",Under Process,No,IELTS Examiner,Yes,"Urdu, English",Research Assistant,"Lab Techniques, Data Analysis",Positive,University Events
5,Nida,Islamabad,Marketing Manager,3,7.875,"Online courses, group study, extensive reading",Master's,30,Singapore,Intermediate,...,"Podcasts, business articles",Pending,No,Digital Marketing,No,"Urdu, English",Marketing Coordinator,"SEO, Content Creation",Neutral,Industry Conferences
6,Bilal,Karachi,Finance Analyst,4,7.000,"Self-study, vocabulary building, speaking prac...",Master's,28,UAE,Intermediate,...,"Financial news, language exchange",Pending,No,CFA Level I,No,"Urdu, English",Financial Analyst,"Financial Modeling, Data Analysis",Neutral,Online Forums
7,Sara,Lahore,Medical Student,5,8.750,"Daily practice, study groups, extensive reading",Bachelor's,23,Ireland,Advanced,...,"Medical journals, language exchange",Approved,No,BLS Certification,Yes,"Urdu, English",Research Assistant,"Lab Techniques, Data Analysis",Positive,Medical Conferences
8,Imran,Islamabad,Software Engineer,3,8.500,"Private tutor, mock tests, listening skill imp...",Bachelor's,26,Germany,Advanced,...,"Coding platforms, language exchange",Under Process,No,ISTQB Certified Tester,Yes,"Urdu, English",Software Developer,"Programming, QA Testing",Positive,Coding Communities
9,Hina,Karachi,Journalist,6,7.500,"Balanced study routine, writing practice, onli...",Master's,31,Malaysia,Intermediate,...,"News articles, language exchange",Rejected,No,Journalism Certification,No,"Urdu, English",Content Writer,"Writing, Editing",Neutral,Freelance Platforms


In [98]:
multiple_features = ["Key Strategies","Preferred Learning Resources","Relevant Skills"]
df_1 = df.copy()

col = df.columns.values
new_col = []
num_col = []
start = 0
for i in multiple_features:
    data = df[i].copy()
    d = np.array(list(map(lambda x: [i.strip() for i in x.split(",")],data)))
    colu = []
    for j in range(d.shape[1]):
        df_1[i+" "+str(j)] = d[:,j]
        colu.append(i+" "+str(j))
    df_1 = df_1.drop(columns=i)
    stop = np.where(col==i)[0][0]
    new_col += col[start:stop].tolist()
    new_col += colu
    num_col += colu
    start = stop + 1
df_2 = df_1[new_col].drop(columns=["Language Fluency"])
df_2["IELTS Score (Overall)"] = df_2["IELTS Score (Overall)"].round(2)

In [99]:
df_3 = df_2.copy()
for i in num_col:
    df_3[i] = df_2[i].str.lower()

In [100]:
key_strategies = []

key_strategies += df_3["Key Strategies 0"].unique().tolist()
key_strategies += df_3["Key Strategies 1"].unique().tolist()
key_strategies += df_3["Key Strategies 2"].unique().tolist()

key_strategies = np.sort(pd.Series(key_strategies).unique()).tolist()


preferred_learning = []

preferred_learning += df_3["Preferred Learning Resources 0"].unique().tolist()
preferred_learning += df_3["Preferred Learning Resources 1"].unique().tolist()

preferred_learning = np.sort(pd.Series(preferred_learning).unique()).tolist()


relevant_skills = []

relevant_skills += df_3["Relevant Skills 0"].unique().tolist()
relevant_skills += df_3["Relevant Skills 1"].unique().tolist()

relevant_skills = np.sort(pd.Series(relevant_skills).unique()).tolist()

In [101]:
df_3

,Candidate,Location,Profession,Study Duration (months),IELTS Score (Overall),Key Strategies 0,Key Strategies 1,Key Strategies 2,Education Level,Age,...,Achieved Desired Score,Preferred Learning Resources 0,Preferred Learning Resources 1,Application Status,Job Offer Received,Additional Certifications,Volunteer Experience,Internship Experience,Relevant Skills 0,Relevant Skills 1
0,Aisha,Karachi,Grad Student,3,8.50,practice,forums,tutoring,Master's,24,...,Yes,online courses,study guides,Under Process,No,TESOL,Yes,Research Assistant,data analysis,teaching
1,Ahmed,Lahore,IT Professional,4,7.50,balanced schedule,time management,speaking practice,Bachelor's,29,...,Yes,online forums,practice tests,Approved,Yes,PMP,No,IT Consultant,project management,programming
2,Fatima,Islamabad,School Teacher,2,9.00,intensive course,online resources,mock interviews,Master's,32,...,Yes,study guides,language exchange,Under Process,No,CELTA,Yes,Curriculum Developer,lesson planning,educational tech
3,Ali,Peshawar,IT Professional,5,8.00,focused study,workshops,peer feedback,Bachelor's,27,...,Yes,mock exams,speaking clubs,Rejected,No,AWS Certified Solutions Architect,No,Software Developer,cloud computing,programming
4,Saad,Lahore,Eng. Student,6,8.25,consistent practice,private tutor,speaking clubs,Bachelor's,22,...,Yes,online tutorials,speaking practice,Under Process,No,IELTS Examiner,Yes,Research Assistant,lab techniques,data analysis
5,Nida,Islamabad,Marketing Manager,3,7.88,online courses,group study,extensive reading,Master's,30,...,No,podcasts,business articles,Pending,No,Digital Marketing,No,Marketing Coordinator,seo,content creation
6,Bilal,Karachi,Finance Analyst,4,7.00,self-study,vocabulary building,speaking practice,Master's,28,...,No,financial news,language exchange,Pending,No,CFA Level I,No,Financial Analyst,financial modeling,data analysis
7,Sara,Lahore,Medical Student,5,8.75,daily practice,study groups,extensive reading,Bachelor's,23,...,Yes,medical journals,language exchange,Approved,No,BLS Certification,Yes,Research Assistant,lab techniques,data analysis
8,Imran,Islamabad,Software Engineer,3,8.50,private tutor,mock tests,listening skill improvement,Bachelor's,26,...,Yes,coding platforms,language exchange,Under Process,No,ISTQB Certified Tester,Yes,Software Developer,programming,qa testing
9,Hina,Karachi,Journalist,6,7.50,balanced study routine,writing practice,online resources,Master's,31,...,No,news articles,language exchange,Rejected,No,Journalism Certification,No,Content Writer,writing,editing


In [102]:
tfidf = TfidfVectorizer()
ppp = tfidf.fit_transform(key_strategies)
qqq = cosine_similarity(ppp).round(2)
dic_list = []
j = 0
i = 0
ctr = 0
for i in range(qqq.shape[0]):
    n = []
    dic = {}
    for j in range(i+1,len(qqq[i])):
        if qqq[i,j] > 0 and qqq[i,j] < 1:
            n.append(j)
    if len(n)>0:
        dic[key_strategies[n[0]]] = key_strategies[i]
    else:
        dic[key_strategies[i]] = key_strategies[i]
    dic_list.append(dic)

df_4 = df_3.copy()
for i in dic_list[::-1]:
    hg = i.copy()
    df_4["Key Strategies 0"] = df_4["Key Strategies 0"].map(hg).fillna(df_4["Key Strategies 0"])
    df_4["Key Strategies 1"] = df_4["Key Strategies 1"].map(hg).fillna(df_4["Key Strategies 1"])
    df_4["Key Strategies 2"] = df_4["Key Strategies 2"].map(hg).fillna(df_4["Key Strategies 2"])

In [103]:
tfidf = TfidfVectorizer()
ppp = tfidf.fit_transform(preferred_learning)
qqq = cosine_similarity(ppp).round(2)
dic_list = []
j = 0
i = 0
ctr = 0
for i in range(qqq.shape[0]):
    n = []
    dic = {}
    for j in range(i+1,len(qqq[i])):
        if qqq[i,j] > 0 and qqq[i,j] < 1:
            n.append(j)
    if len(n)>0:
        dic[preferred_learning[n[0]]] = preferred_learning[i]
    else:
        dic[preferred_learning[i]] = preferred_learning[i]
    dic_list.append(dic)

df_5 = df_4.copy()
for i in dic_list[::-1]:
    hg = i.copy()
    df_5["Preferred Learning Resources 0"] = df_5["Preferred Learning Resources 0"].map(hg).fillna(df_5["Preferred Learning Resources 0"])
    df_5["Preferred Learning Resources 1"] = df_5["Preferred Learning Resources 1"].map(hg).fillna(df_5["Preferred Learning Resources 1"])

In [104]:
tfidf = TfidfVectorizer()
ppp = tfidf.fit_transform(relevant_skills)
qqq = cosine_similarity(ppp).round(2)
dic_list = []
j = 0
i = 0
ctr = 0
for i in range(qqq.shape[0]):
    n = []
    dic = {}
    for j in range(i+1,len(qqq[i])):
        if qqq[i,j] > 0 and qqq[i,j] < 1:
            n.append(j)
    if len(n)>0:
        dic[relevant_skills[n[0]]] = relevant_skills[i]
    else:
        dic[relevant_skills[i]] = relevant_skills[i]
    dic_list.append(dic)

df_6 = df_5.copy()
for i in dic_list[::-1]:
    hg = i.copy()
    df_6["Relevant Skills 0"] = df_6["Relevant Skills 0"].map(hg).fillna(df_6["Relevant Skills 0"])
    df_6["Relevant Skills 1"] = df_6["Relevant Skills 1"].map(hg).fillna(df_6["Relevant Skills 1"])

In [105]:
df_6

,Candidate,Location,Profession,Study Duration (months),IELTS Score (Overall),Key Strategies 0,Key Strategies 1,Key Strategies 2,Education Level,Age,...,Achieved Desired Score,Preferred Learning Resources 0,Preferred Learning Resources 1,Application Status,Job Offer Received,Additional Certifications,Volunteer Experience,Internship Experience,Relevant Skills 0,Relevant Skills 1
0,Aisha,Karachi,Grad Student,3,8.50,intensive course,forums,tutoring,Master's,24,...,Yes,online courses,study guides,Under Process,No,TESOL,Yes,Research Assistant,data analysis,teaching
1,Ahmed,Lahore,IT Professional,4,7.50,balanced schedule,time management,speaking clubs,Bachelor's,29,...,Yes,online courses,practice tests,Approved,Yes,PMP,No,IT Consultant,project management,programming
2,Fatima,Islamabad,School Teacher,2,9.00,intensive course,online courses,mock interviews,Master's,32,...,Yes,study guides,language exchange,Under Process,No,CELTA,Yes,Curriculum Developer,lesson planning,educational tech
3,Ali,Peshawar,IT Professional,5,8.00,balanced schedule,workshops,peer feedback,Bachelor's,27,...,Yes,mock exams,speaking clubs,Rejected,No,AWS Certified Solutions Architect,No,Software Developer,cloud computing,programming
4,Saad,Lahore,Eng. Student,6,8.25,consistent practice,private tutor,speaking clubs,Bachelor's,22,...,Yes,online courses,speaking clubs,Under Process,No,IELTS Examiner,Yes,Research Assistant,lab techniques,data analysis
5,Nida,Islamabad,Marketing Manager,3,7.88,online courses,balanced schedule,extensive reading,Master's,30,...,No,podcasts,business articles,Pending,No,Digital Marketing,No,Marketing Coordinator,seo,content creation
6,Bilal,Karachi,Finance Analyst,4,7.00,balanced schedule,vocabulary building,speaking clubs,Master's,28,...,No,financial news,language exchange,Pending,No,CFA Level I,No,Financial Analyst,3d modeling,data analysis
7,Sara,Lahore,Medical Student,5,8.75,consistent practice,balanced schedule,extensive reading,Bachelor's,23,...,Yes,engineering journals,language exchange,Approved,No,BLS Certification,Yes,Research Assistant,lab techniques,data analysis
8,Imran,Islamabad,Software Engineer,3,8.50,private tutor,mock interviews,listening skill improvement,Bachelor's,26,...,Yes,coding platforms,language exchange,Under Process,No,ISTQB Certified Tester,Yes,Software Developer,programming,qa testing
9,Hina,Karachi,Journalist,6,7.50,balanced schedule,writing practice,online courses,Master's,31,...,No,financial news,language exchange,Rejected,No,Journalism Certification,No,Content Writer,writing,editing
